In [1]:
import pandas as pd
import numpy as np
import time
import datetime as dt

from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor

from sklearn.model_selection import GridSearchCV

from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error


c:\ProgramData\anaconda3\envs\tripleenv\lib\site-packages\sklearn\utils\fixes.py:28: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import parse_version  # type: ignore


In [2]:
df = pd.read_csv('C:/Users/hueso/Nextcloud/Personal_L/Tripleten/Course/Sprint_14/File/car_data.csv')
df.info()
df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   DateCrawled        354369 non-null  object
 1   Price              354369 non-null  int64 
 2   VehicleType        316879 non-null  object
 3   RegistrationYear   354369 non-null  int64 
 4   Gearbox            334536 non-null  object
 5   Power              354369 non-null  int64 
 6   Model              334664 non-null  object
 7   Mileage            354369 non-null  int64 
 8   RegistrationMonth  354369 non-null  int64 
 9   FuelType           321474 non-null  object
 10  Brand              354369 non-null  object
 11  NotRepaired        283215 non-null  object
 12  DateCreated        354369 non-null  object
 13  NumberOfPictures   354369 non-null  int64 
 14  PostalCode         354369 non-null  int64 
 15  LastSeen           354369 non-null  object
dtypes: int64(7), object(

,Price,RegistrationYear,Power,Mileage,RegistrationMonth,NumberOfPictures,PostalCode
count,354369.000000,354369.000000,354369.000000,354369.000000,354369.000000,354369.0,354369.000000
mean,4416.656776,2004.234448,110.094337,128211.172535,5.714645,0.0,50508.689087
std,4514.158514,90.227958,189.850405,37905.341530,3.726421,0.0,25783.096248
min,0.000000,1000.000000,0.000000,5000.000000,0.000000,0.0,1067.000000
25%,1050.000000,1999.000000,69.000000,125000.000000,3.000000,0.0,30165.000000
50%,2700.000000,2003.000000,105.000000,150000.000000,6.000000,0.0,49413.000000
75%,6400.000000,2008.000000,143.000000,150000.000000,9.000000,0.0,71083.000000
max,20000.000000,9999.000000,20000.000000,150000.000000,12.000000,0.0,99998.000000


In [3]:
df.sample(5)

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Mileage,RegistrationMonth,FuelType,Brand,NotRepaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
90300,04/04/2016 23:56,16500,convertible,2006,auto,163,slk,60000,11,petrol,mercedes_benz,no,04/04/2016 00:00,0,39114,07/04/2016 09:16
345473,05/04/2016 11:45,1850,sedan,2003,auto,144,other,150000,2,lpg,opel,no,05/04/2016 00:00,0,40472,05/04/2016 11:45
284783,29/03/2016 18:58,1000,sedan,1996,manual,136,vectra,125000,6,petrol,opel,no,29/03/2016 00:00,0,63450,06/04/2016 04:46
107923,11/03/2016 13:25,3250,small,2005,manual,69,fiesta,80000,10,petrol,ford,no,11/03/2016 00:00,0,56470,26/03/2016 10:44
97177,02/04/2016 18:53,650,sedan,1996,manual,90,golf,150000,6,gasoline,volkswagen,no,02/04/2016 00:00,0,38820,06/04/2016 19:44


### Preparación de datos

Viendo los datos como están ahorita procederé a eliminar duplicados, cambiar a tipo de datos correcto, verificar que caracteristicas son relevantes y revisar si es pertinente remover NaNs o modificar esos datos.

In [4]:
df.duplicated().sum()

262

In [5]:
df.drop_duplicates(inplace=True)
df.duplicated().sum()

0

In [6]:
df['VehicleType'].value_counts()

VehicleType
sedan          91399
small          79753
wagon          65115
bus            28752
convertible    20180
coupe          16147
suv            11991
other           3286
Name: count, dtype: int64

In [7]:
df['VehicleType'].fillna('other',inplace=True)

C:\Users\hueso\AppData\Local\Temp\ipykernel_22752\535524898.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['VehicleType'].fillna('other',inplace=True)


In [8]:
df['VehicleType'].isna().sum()

0

En tipo de vehiculo los datos faltantes en ves de quitarlos, pueden ser considerados dentro de la categoria de "other" para asi mantener las demás caracteristicas de estas observaciones.

In [9]:
df['Gearbox'].value_counts()

Gearbox
manual    268034
auto       66243
Name: count, dtype: int64

In [10]:
df['Gearbox'].isna().sum()

19830

In [ ]:
df.dropna(subset=['Gearbox'],inplace=True)
df['Gearbox'].isna().sum()

0

En la caracteristica "Gearbox" no puedo determinar si los datos faltantes son de una u otra categoria, asi que decidí eliminarlos.

In [12]:
df['Model'].value_counts()

Model
golf                  27740
other                 23007
3er                   19069
polo                  12325
corsa                 11780
                      ...  
i3                        7
rangerover                4
serie_3                   3
range_rover_evoque        2
serie_1                   1
Name: count, Length: 250, dtype: int64

In [13]:
df['Model']=df['Model'].fillna('other')
df['Model'].isna().sum()

0

Los modelos que son NaN pueden catalogarse como "other", ya que anteriormente la categoria ya existía.

In [14]:
df['RegistrationMonth'].value_counts()

RegistrationMonth
3     33425
6     30429
4     28399
5     28240
0     27580
7     26357
10    25263
12    23554
9     23096
11    23092
1     22297
8     21892
2     20653
Name: count, dtype: int64

In [15]:
df.drop(df[df['RegistrationMonth']==0].index,inplace=True)

Eliminé los valores con mes 0 ya que este no existe.

In [16]:
df['FuelType'].value_counts()

FuelType
petrol      193002
gasoline     91008
lpg           4807
cng            525
hybrid         223
other          125
electric        71
Name: count, dtype: int64

In [17]:
df['FuelType']=df['FuelType'].fillna('other')
df['FuelType'].isna().sum()

0

Los valores NaN pueden renombrarse a other, ya que anteriormente existía la categoria de "other"

In [18]:
df['NotRepaired'].value_counts()

NotRepaired
no     232151
yes     29882
Name: count, dtype: int64

In [ ]:
df.dropna(subset=['NotRepaired'],inplace=True)
df['NotRepaired'].isna().sum()

KeyError: ['NotRepaired']

In [20]:
df['NumberOfPictures'].value_counts()

NumberOfPictures
0    262033
Name: count, dtype: int64

In [21]:
df.drop(['DateCrawled'],axis=1,inplace=True)

Eliminé la caracteristica DateCrawled ya que no es relevante al ser solo unas horas despues de DateCreated. 

In [22]:
df.drop(['NumberOfPictures'],axis=1,inplace=True)

El numero de imagenes es 0 en todas las observaciones. Al ser un dato que no cambia, no nos sirve y nos complicaria los modelos. Mejor lo eliminé.
Todos las caracteristicas que son númericas no tienen valores faltantes, por lo que no fue necesario tratar NaNs.
Ahora pondré los tipos de dato correctos para cada columna.

In [23]:
df['DateCreated'] = pd.to_datetime(df['DateCreated'], format='%d/%m/%Y %H:%M')
df['LastSeen'] = pd.to_datetime(df['LastSeen'], format='%d/%m/%Y %H:%M')

Ahora solo cambiaré las columnas a que tengan el nombre con "snake_case" y revisaré como quedaron los datos.

In [24]:
df.columns = df.columns.str.replace(r'([a-z0-9])([A-Z])', r'\1_\2', regex=True).str.lower()
df.reset_index(drop=True,inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 262033 entries, 0 to 262032
Data columns (total 14 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   price               262033 non-null  int64         
 1   vehicle_type        262033 non-null  object        
 2   registration_year   262033 non-null  int64         
 3   gearbox             262033 non-null  object        
 4   power               262033 non-null  int64         
 5   model               262033 non-null  object        
 6   mileage             262033 non-null  int64         
 7   registration_month  262033 non-null  int64         
 8   fuel_type           262033 non-null  object        
 9   brand               262033 non-null  object        
 10  not_repaired        262033 non-null  object        
 11  date_created        262033 non-null  datetime64[ns]
 12  postal_code         262033 non-null  int64         
 13  last_seen           262033 no

Para poder mejorar los datos a que sean más usables para ML podemos mapear algunos strings y fechas a int.

In [25]:
df['not_repaired'] = df['not_repaired'].map({"yes":1, "no":0}).astype('int8')
df['date_created']=df['date_created'].map(dt.datetime.toordinal)
df['last_seen']=df['last_seen'].map(dt.datetime.toordinal)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 262033 entries, 0 to 262032
Data columns (total 14 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   price               262033 non-null  int64 
 1   vehicle_type        262033 non-null  object
 2   registration_year   262033 non-null  int64 
 3   gearbox             262033 non-null  object
 4   power               262033 non-null  int64 
 5   model               262033 non-null  object
 6   mileage             262033 non-null  int64 
 7   registration_month  262033 non-null  int64 
 8   fuel_type           262033 non-null  object
 9   brand               262033 non-null  object
 10  not_repaired        262033 non-null  int8  
 11  date_created        262033 non-null  int64 
 12  postal_code         262033 non-null  int64 
 13  last_seen           262033 non-null  int64 
dtypes: int64(8), int8(1), object(5)
memory usage: 26.2+ MB


Los datos ya están listos para ser usados en ML

### Entrenamiento de los modelos

Primero haré una regresión lineal para determinar si los siguientes modelos con busqueda de mejores hiperparametros por medio de GridSearchCV funcionarán mejor que este modelo. De no tener un mejor RMSE algo está mal en el codigo.

#### Linear Regression (Cordura)

In [26]:
numeric=['registration_year', 'power','mileage','last_seen']
df_lr=df.copy()
scaler=StandardScaler()
scaler.fit(df_lr[numeric])
df_lr[numeric]=scaler.transform(df_lr[numeric])

In [27]:
#df_lr.info()

In [28]:
features = df_lr.drop('price',axis=1)
target = df_lr['price']

In [29]:
features_ohe = pd.get_dummies(features,drop_first=True)

In [30]:
features_ohe.sample()

,registration_year,power,mileage,registration_month,not_repaired,date_created,postal_code,last_seen,vehicle_type_convertible,vehicle_type_coupe,...,brand_seat,brand_skoda,brand_smart,brand_sonstige_autos,brand_subaru,brand_suzuki,brand_toyota,brand_trabant,brand_volkswagen,brand_volvo
104767,-0.720793,-0.199848,0.609472,4,0,736035,85540,-1.401,False,False,...,False,False,False,False,False,False,False,False,False,False


In [31]:
%%time

features_train, features_test, target_train, target_test = train_test_split(features_ohe, target, test_size=0.25, random_state=123)

model = LinearRegression()
model.fit(features_train,target_train)
target_pred = model.predict(features_test)
rmse = mean_squared_error(target_test,target_pred)**0.5
print (f"El Error Cuadratico Medio es: {rmse:.4f}")
print ('Tiempo de ejecución:')

El Error Cuadratico Medio es: 4274.0319
Tiempo de ejecución:
CPU times: total: 15 s
Wall time: 3.46 s


Para conseguir los mejores valores hiperparametricos usaré GridSearchCV en cada uno de los modelos por medio de una función.

In [32]:
def gsearch(model,model_name,search_space):

    model = model
    model_grid = GridSearchCV(model,param_grid=search_space,cv=3,scoring='neg_root_mean_squared_error',verbose=2,n_jobs=-1)

    model_grid.fit(features_train,target_train)

    best_params = model_grid.best_params_
    best_score = model_grid.best_score_
    best_grid = model_grid.best_estimator_

    predicted_values = best_grid.predict(features_test)

    RMSE = mean_squared_error(target_test,predicted_values)**0.5
    
    print("Best Parameters For", model_name,":", best_params )
    print("These Parameters Produced a Best RMSE Score of", round(best_score,2), "on the Training Data")
    print()
    print("The RMSE Using the Tuned", model_name, "on the Validation Set:", round(RMSE,2))
    #print("Best grid:", best_grid )
    print()
    print('Training Time for', model_name,':')



#### Random Forest Regressor

In [44]:
%%time
df_rf = df.copy()

features = df_rf.drop('price',axis=1)
target = df_rf['price']

features_ohe = pd.get_dummies(features,drop_first=True)

features_train, features_test, target_train, target_test = train_test_split(features_ohe, target, test_size=0.25, random_state=123)

model = RandomForestRegressor(verbose=1,random_state=123,n_jobs=-1)
model_name = 'RandomForestRegressor'
search_space = {'n_estimators':[5,10,100], 'max_depth':[5,10,20],'min_samples_split': [5, 10]}

gsearch(model,model_name,search_space)

Fitting 3 folds for each of 18 candidates, totalling 54 fits


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 20 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:    7.2s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:   37.4s finished


Best Parameters For RandomForestRegressor : {'max_depth': 20, 'min_samples_split': 5, 'n_estimators': 100}
These Parameters Produced a Best RMSE Score of -1750.14 on the Training Data

The RMSE Using the Tuned RandomForestRegressor on the Validation Set: 1705.64

Training Time for RandomForestRegressor :
CPU times: total: 11min 31s
Wall time: 7min 12s


[Parallel(n_jobs=20)]: Using backend ThreadingBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done  10 tasks      | elapsed:    0.0s
[Parallel(n_jobs=20)]: Done 100 out of 100 | elapsed:    0.0s finished


### LightGBM

In [49]:
categorical_cols=['vehicle_type','gearbox','model','fuel_type','brand']
df_lgbm = df.copy()

for col in categorical_cols:
    df_lgbm[col] = df_lgbm[col].astype('category')

features = df_lgbm.drop('price',axis=1)
target = df_lgbm['price']

features_train, features_test, target_train, target_test = train_test_split(features, target, test_size=0.25, random_state=123)

In [46]:
%%time

modellgbm = LGBMRegressor(random_state=123)
model_name = "LightGBMRegressor"
search_space = {'n_estimators': [5,10,100], 'max_depth':[5,10,20],'learning_rate': [0.01, 0.03,.1]}

gsearch(modellgbm, model_name,search_space)


Fitting 3 folds for each of 27 candidates, totalling 81 fits


c:\ProgramData\anaconda3\envs\tripleenv\lib\site-packages\joblib\externals\loky\process_executor.py:782: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002178 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1038
[LightGBM] [Info] Number of data points in the train set: 196524, number of used features: 13
[LightGBM] [Info] Start training from score 5061.232221
Best Parameters For LightGBMRegressor : {'learning_rate': 0.1, 'max_depth': 10, 'n_estimators': 100}
These Parameters Produced a Best RMSE Score of -1718.83 on the Training Data

The RMSE Using the Tuned LightGBMRegressor on the Validation Set: 1703.99

Training Time for LightGBMRegressor :
CPU times: total: 5.25 s
Wall time: 11.3 s


### CatBoost

In [51]:
%%time
model = CatBoostRegressor(random_state=123,cat_features=categorical_cols)
model_name  = 'CatBoostRegressor'
search_space = {'iterations':[5,10,100], 'depth':[5,10,15],'learning_rate': [0.01, 0.03,.1]}

gsearch(model,model_name,search_space)

Fitting 3 folds for each of 27 candidates, totalling 81 fits
0:	learn: 4367.0939500	total: 418ms	remaining: 41.4s
1:	learn: 4059.5805342	total: 835ms	remaining: 40.9s
2:	learn: 3792.8463556	total: 1.26s	remaining: 40.8s
3:	learn: 3555.8701207	total: 1.69s	remaining: 40.5s
4:	learn: 3348.1769983	total: 2.08s	remaining: 39.6s
5:	learn: 3170.0967740	total: 2.48s	remaining: 38.9s
6:	learn: 3013.7296025	total: 2.88s	remaining: 38.2s
7:	learn: 2876.3150037	total: 3.28s	remaining: 37.7s
8:	learn: 2751.4047125	total: 3.71s	remaining: 37.6s
9:	learn: 2645.5436664	total: 4.12s	remaining: 37.1s
10:	learn: 2536.8507658	total: 4.51s	remaining: 36.5s
11:	learn: 2440.2110261	total: 4.92s	remaining: 36.1s
12:	learn: 2355.9346382	total: 5.3s	remaining: 35.5s
13:	learn: 2280.8833334	total: 5.69s	remaining: 35s
14:	learn: 2218.7646124	total: 6.13s	remaining: 34.7s
15:	learn: 2161.9249866	total: 6.52s	remaining: 34.2s
16:	learn: 2113.5802249	total: 6.91s	remaining: 33.8s
17:	learn: 2071.8762237	total: 7.3

### Conclusion de Entrenamiento de modelos

Se usaron tres modelos distintos Random Forest Regressor, LightGBM y CatBoost con casi los mismos hiperparametros para que pudiera ser comparable el tiempo que tarda cada uno de los modelos.

Tiempos (CPU Times en mi PC):
RandomForestRegressor: 11min 31seg RMSE: 1705.64
LightGbm: 5.25s RMSE: 1703.99
CatBoost: 2min 51s RMSE: 1687.95

A partir de aqui podemos darnos una idea de que modelo nos da el mejor resultado en el menor tiempo posible. Todos dan resultados relativamente cercanos entre si, sin embargo los tiempos varian considerablemente, siendo LightGbm el que yo seleccionaria en este momento, pero para poder determinar que modelo es el mejor para Rusty Bargain ahora procederé a entrenar los 3 modelos anteriores pero solo con los mejores hiperparametros que encontró GridSearhCV para cada uno.



### Analisis del Modelo

#### Random Forest Regressor

In [70]:
%%time
df_rf = df.copy()

features = df_rf.drop('price',axis=1)
target = df_rf['price']

features_ohe = pd.get_dummies(features,drop_first=True)

features_train, features_test, target_train, target_test = train_test_split(features_ohe, target, test_size=0.25, random_state=123)

model_rf = RandomForestRegressor(max_depth=20,min_samples_leaf=5,n_estimators=100,verbose=0,random_state=123,n_jobs=-1)
model_rf.fit(features_train,target_train)

predicted_values = model_rf.predict(features_test)

RMSE = mean_squared_error(target_test,predicted_values)**0.5
    
print(f'El error RMSE es de: {RMSE}')
print('Los tiempos de ejecución son:')


El error RMSE es de: 1730.7949359739973
Los tiempos de ejecución son:
CPU times: total: 11min 38s
Wall time: 43 s


#### LightGBM

In [71]:
%%time
categorical_cols=['vehicle_type','gearbox','model','fuel_type','brand']
df_lgbm = df.copy()

for col in categorical_cols:
    df_lgbm[col] = df_lgbm[col].astype('category')

features = df_lgbm.drop('price',axis=1)
target = df_lgbm['price']

features_train, features_test, target_train, target_test = train_test_split(features, target, test_size=0.25, random_state=123)

modellgbm = LGBMRegressor(learning_rate=.1,max_depth=10,n_estimators=100,random_state=123,verbose=0,n_jobs=-1)
modellgbm.fit(features_train,target_train)

predicted_values = modellgbm.predict(features_test)

RMSE = mean_squared_error(target_test,predicted_values)**0.5
    
print(f'El error RMSE es de: {RMSE}')
print('Los tiempos de ejecución son:')

El error RMSE es de: 1703.9942194640937
Los tiempos de ejecución son:
CPU times: total: 5.52 s
Wall time: 437 ms


#### CatBoost

In [72]:
%%time
categorical_cols=['vehicle_type','gearbox','model','fuel_type','brand']
df_cb = df.copy()

for col in categorical_cols:
    df_cb[col] = df_cb[col].astype('category')

features = df_cb.drop('price',axis=1)
target = df_cb['price']

features_train, features_test, target_train, target_test = train_test_split(features, target, test_size=0.25, random_state=123)

modelcb = CatBoostRegressor(cat_features=categorical_cols,depth=15,iterations=100,learning_rate=.1,random_state=123,verbose=0)
modelcb.fit(features_train,target_train)

predicted_values = modelcb.predict(features_test)

RMSE = mean_squared_error(target_test,predicted_values)**0.5
    
print(f'El error RMSE es de: {RMSE}')
print('Los tiempos de ejecución son:')

El error RMSE es de: 1687.9511984177454
Los tiempos de ejecución son:
CPU times: total: 2min 41s
Wall time: 38.3 s


Despues de entrenar los modelos con los mejores hiperparametros encontrados puedo recomendar a Rusty Bargain que se use LightGBM por dar el un muy buen resultado en el menor tiempo posible o si se prefiere invertir un poco más de tiempo CatBoost nos puede dar un mejor resultado.